In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string

random.seed(134)

PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32

train = 'data/snli_train.tsv'
val = 'data/snli_val.tsv'

In [2]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [words2id[word] if word in words2id else UNK_IDX for word in tokens]
        indices_data.append(index_list)
    return indices_data

In [3]:
words2id = pkl.load(open("data/words2id.pkl", "rb"))
idx2words = pkl.load(open("data/idx2words.pkl", "rb"))
loaded_embeddings = pkl.load(open("data/embedding_matrix.pkl", "rb"))

train_data = pd.read_csv(train,delimiter='\t',encoding='utf-8')
val_data = pd.read_csv(val,delimiter='\t',encoding='utf-8')

train_data['label_num'] = train_data['label'].apply(lambda x: 0 if str(x) == 'contradiction' else 1 if str(x) == 'neutral' else 2)
val_data['label_num'] = val_data['label'].apply(lambda x: 0 if str(x) == 'contradiction' else 1 if str(x) == 'neutral' else 2)

train_label = list(train_data['label_num'])
val_label = list(val_data['label_num'])

In [4]:
train_tokens_1 = pkl.load(open("data/train_data_tokens_1.p", "rb"))
train_tokens_2 = pkl.load(open("data/train_data_tokens_2.p", "rb"))

val_tokens_1 = pkl.load(open("data/val_data_tokens_1.p", "rb"))
val_tokens_2 = pkl.load(open("data/val_data_tokens_2.p", "rb"))

In [5]:
train_data_indices_1 = token2index_dataset(train_tokens_1)
train_data_indices_2 = token2index_dataset(train_tokens_2)
val_data_indices_1 = token2index_dataset(val_tokens_1)
val_data_indices_2 = token2index_dataset(val_tokens_2)

In [6]:
MAX_SENTENCE_LENGTH = 200
BATCH_SIZE = 32

class VocabDataset(Dataset):
    """
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, data_list_1, data_list_2, target_list, words2id):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.data_1 = data_list_1
        self.data_2 = data_list_2
        self.target_list = target_list
        assert (len(self.data_1) == len(self.target_list))
        assert (len(self.data_2) == len(self.target_list))
        self.words2id = words2id

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        words_idx_1 = self.data_1[key][:MAX_SENTENCE_LENGTH]
        
        words_idx_2 = self.data_2[key][:MAX_SENTENCE_LENGTH]
        
        label = self.target_list[key]
        return [words_idx_1, len(words_idx_1), words_idx_2,len(words_idx_2),label]

def vocab_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    data_list_1 = []
    data_list_2 = []
    label_list = []
    length_list_1 = []
    length_list_2 = []

    for datum in batch:
        label_list.append(datum[4])
        length_list_1.append(datum[1])
        length_list_2.append(datum[3])
        
    # padding
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])),
                                mode="constant", constant_values=0)
        data_list_1.append(padded_vec_1)
        
        padded_vec_2 = np.pad(np.array(datum[2]),
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[3])),
                                mode="constant", constant_values=0)
        data_list_2.append(padded_vec_2)
        

    #handle torch type problem by adding the following line
    data_list_2 = np.asarray(data_list_2, dtype=int)
    label_list = np.array(label_list)
    
    return [torch.from_numpy(np.array(data_list_1)), torch.LongTensor(length_list_1), 
            torch.from_numpy(np.array(data_list_2)), torch.LongTensor(length_list_2),
            torch.LongTensor(label_list)]


In [7]:
# Build train and valid dataloaders

train_dataset = VocabDataset(train_data_indices_1, train_data_indices_2, train_label,words2id)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)


val_dataset = VocabDataset(val_data_indices_1,val_data_indices_2,val_label, words2id)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)



In [46]:
def generate_weights_matrix(idx2words,loaded_embeddings):
   
    matrix_len = len(idx2words)
    weights_matrix = np.zeros((matrix_len, 300))
    
    for key in idx2words.keys():
        try: 
            weights_matrix[key] = loaded_embeddings[key]
        except KeyError:
            weights_matrix[key] = np.random.normal(scale=0.6, size=(emb_dim, ))
    return weights_matrix

In [10]:
weights_matrix = generate_weights_matrix(idx2words,loaded_embeddings)

In [95]:
class RNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers, num_classes):
        # RNN Accepts the following hyperparams:
        # hidden_size: Hidden Size of layer in RNN
        # num_layers: number of layers in RNN
        # vocab_size: vocabulary size
        super(RNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        num_embeddings, embedding_dim = weights_matrix.shape
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(weights_matrix))
        self.embedding.weight.requires_grad = False
        
        self.bi_gru = nn.GRU(embedding_dim, hidden_size, num_layers=1, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(2 * hidden_size, 80)
        self.fc2 = nn.Linear(80, num_classes)

    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        #biased term?
        hidden = torch.randn(self.num_layers * 2 , batch_size, self.hidden_size)

        return hidden
    
    
    def forward(self, x, lengths_x, y,lengths_y):
        #sort the data by length descending order
        seq_lengths_x, perm_idx_x = lengths_x.sort(0,descending=True)
        x = x[perm_idx_x]
        lengths_x = lengths_x[perm_idx_x]


        seq_lengths_y, perm_idx_y = lengths_y.sort(0,descending=True)
        y = y[perm_idx_y]
        lengths_y = lengths_y[perm_idx_y]
        
        # reset hidden state
        batch_size_x = x.size()[0]
        self.hidden_x = self.init_hidden(batch_size_x)
        
        # get embedding of characters
        embed_x = self.embedding(x)       
        embed_x = torch.nn.utils.rnn.pack_padded_sequence(embed_x, lengths_x.numpy(), batch_first=True)
        rnn_out_x, (ht_x, ct) = self.bi_gru(embed_x,self.hidden_x)
        rnn_out_x, _ = torch.nn.utils.rnn.pad_packed_sequence(rnn_out_x, batch_first=True)
        
        rnn_out_x = rnn_out_x[:, -1, :self.hidden_size] + rnn_out_x[:, 0, self.hidden_size:] 
        #rnn_out_x = rnn_out_x[:, -1, :self.hidden_size]
        decode = ht_x[-1]
        
        batch_size_y = y.size()[0]
        self.hidden_y = self.init_hidden(batch_size_y)
        
        embed_y = self.embedding(y)
        embed_y = torch.nn.utils.rnn.pack_padded_sequence(embed_y, lengths_y.numpy(), batch_first=True)
        rnn_out_y, (ht_y, ct) = self.bi_gru(embed_y, self.hidden_y)
        rnn_out_y, _ = torch.nn.utils.rnn.pad_packed_sequence(rnn_out_y, batch_first=True)
        
        rnn_out_y = rnn_out_y[:, -1, :self.hidden_size] + rnn_out_y[:, 0, self.hidden_size:]
#        rnn_out_y = rnn_out_y[:, -1, :]
  #      out = ht[-2:].transpose(0, 1).contiguous().view(batch_size_y, -1)
        
        run_out = torch.cat([rnn_out_x, rnn_out_y], 1)
        
        out = self.dropout(run_out)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        
        logits = F.relu(self.fc1(run_out))
        logits = self.dropout(self.fc2(logits))
        
        return logits


In [100]:
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data_1, lengths_1, data_2, lengths_2, labels in loader:
        
        outputs = F.softmax(model(data_1, lengths_1,data_2, lengths_2), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]

        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).double().sum().item()
    return (100 * correct / total)


model = RNN(weights_matrix, hidden_size=300, num_layers=1, num_classes = 3)
learning_rate = 3e-4
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



for epoch in range(2):
    for i, (data_1, lengths_1, data_2, lengths_2, labels) in enumerate(train_loader):
 
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data_1, lengths_1,data_2,lengths_2)
        
#         if i >= 1:
#             break
        loss = criterion(outputs, labels)

#         # Backward and optimize
        loss.backward()

        optimizer.step()
#         # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))

Epoch: [1/10], Step: [101/3125], Validation Acc: 32.5
Epoch: [1/10], Step: [201/3125], Validation Acc: 34.3
Epoch: [1/10], Step: [301/3125], Validation Acc: 34.6
Epoch: [1/10], Step: [401/3125], Validation Acc: 33.3
Epoch: [1/10], Step: [501/3125], Validation Acc: 33.2


KeyboardInterrupt: 

In [16]:
for i, (data_1, lengths_1, data_2, lengths_2, labels) in enumerate(train_loader):
    
    if i >= 1:
        break
    
    # Sort by length (keep idx)
    print(data_1)
#     print(lengths_1)
    sent_len, idx_sort = np.sort(lengths_1)[::-1], np.argsort(-lengths_1)
    idx_unsort = np.argsort(idx_sort)
    data_1_sort = data_1[idx_sort]
    lengths_sort = lengths_1[idx_sort]

    #idx_sort = torch.from_numpy(idx_sort)
#     print(data_1_sort)
#     print(lengths_sort)
    
    data_unsort = data_1_sort[idx_unsort]
    lenghs_unsort = lengths_sort[idx_unsort]
#     print(data_unsort)
#     print(lenghs_unsort)
    print(data_unsort)
    #sent_variable = sent_variable.index_select(0, Variable(idx_sort))

tensor([[   9,  347,  994,  ...,    0,    0,    0],
        [   9,  454, 1830,  ...,    0,    0,    0],
        [   9,  347,    1,  ...,    0,    0,    0],
        ...,
        [ 309,  249, 5025,  ...,    0,    0,    0],
        [   9,  994, 3466,  ...,    0,    0,    0],
        [   3,  876, 2735,  ...,    0,    0,    0]])
tensor([[   9,  347,  994,  ...,    0,    0,    0],
        [   9,  454, 1830,  ...,    0,    0,    0],
        [   9,  347,    1,  ...,    0,    0,    0],
        ...,
        [ 309,  249, 5025,  ...,    0,    0,    0],
        [   9,  994, 3466,  ...,    0,    0,    0],
        [   3,  876, 2735,  ...,    0,    0,    0]])
